# Graph Convolutional Network by Kipf and Welling

## Imports

In [92]:
import dgl
import dgl.function as fn
import torch as th
import torch.nn as nn
import torch.nn.functional as F
from dgl import DGLGraph

import pickle

In [10]:
class LinearModule(nn.Module):
    """The linear transformation part of the GCN layer"""
    def __init__(self, in_feats, out_feats, activation):
        super(LinearModule, self).__init__()
        self.linear = nn.Linear(in_feats, out_feats)
        self.activation = activation # This is the activation function

    def forward(self, node):
        h = self.linear(node.data['h'])
        h = self.activation(h)
        return {'h' : h}

In [11]:
class GCN(nn.Module):
    """A GCN layer"""
    def __init__(self, in_feats, out_feats, activation):
        super(GCN, self).__init__()
        self.apply_mod = LinearModule(in_feats, out_feats, activation)

    def forward(self, g, feature):
        g.ndata['h'] = feature
        g.update_all(message_func=fn.copy_src(src='h', out='m'), reduce_func=fn.sum(msg='m', out='h'))
        g.apply_nodes(func=self.apply_mod)
        return g.ndata.pop('h')

In [62]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.gcn1 = GCN(1433, 16, F.relu)
        self.gcn2 = GCN(16, 7, lambda x: F.log_softmax(x,1))
        self.dropout = nn.Dropout(0.2)

    def forward(self, g, features):
        x = self.gcn1(g, features)
        x = self.dropout(x)
        x = self.gcn2(g, x)
        return x

In [15]:
from dgl.data import citation_graph as citegrh
import networkx as nx
def load_cora_data():
    data = citegrh.load_cora()
    features = th.FloatTensor(data.features)
    labels = th.LongTensor(data.labels)
    mask = th.ByteTensor(data.train_mask)
    g = data.graph

    # add self loop
    g.remove_edges_from(nx.selfloop_edges(g))
    g = DGLGraph(g)
    g.add_edges(g.nodes(), g.nodes())
    
    return g, features, labels, mask

## Select Training Set

In [98]:
percentage_train = 0.02

with open("data/cora_permutation1.pickle","rb") as f:
    perm1 = pickle.load(f)
mask = np.zeros(g.number_of_nodes())
mask[perm1[range(int(percentage_train*g.number_of_nodes()))]] = 1
mask = th.ByteTensor(mask)

In [99]:
import time
import numpy as np

net = Net()
#print(net)

optimizer = th.optim.Adam(net.parameters(), lr=1e-3)
net.train() # Set to training mode (use dropout)

dur = []
for epoch in range(90):
    if epoch >=3:
        t0 = time.time()

    # Compute loss for test nodes (only for validation, not used by optimizer)
    net.eval()
    prediction = net(g, features)
    val_loss = F.nll_loss(prediction.detach()[1-mask], labels[1-mask])
    net.train()

    # Compute loss for train nodes
    logits = net(g, features)
    loss = F.nll_loss(logits[mask], labels[mask])
    
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    if epoch >=3:
        dur.append(time.time() - t0)

    print("Epoch {:05d} | Loss {:.4f} | Val.Loss {:.4f} | Time(s) {:.4f}".format(
            epoch, loss.item(), val_loss.item(), np.mean(dur)))

C:\Users\a_liso02\AppData\Local\Continuum\anaconda3\lib\site-packages\numpy\core\fromnumeric.py:3118: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
C:\Users\a_liso02\AppData\Local\Continuum\anaconda3\lib\site-packages\numpy\core\_methods.py:85: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
Epoch 00000 | Loss 1.9274 | Val.Loss 1.9411 | Time(s) nan
Epoch 00001 | Loss 1.8792 | Val.Loss 1.9257 | Time(s) nan
Epoch 00002 | Loss 1.8468 | Val.Loss 1.9139 | Time(s) nan
Epoch 00003 | Loss 1.8213 | Val.Loss 1.9037 | Time(s) 0.1520
Epoch 00004 | Loss 1.7917 | Val.Loss 1.8939 | Time(s) 0.1800
Epoch 00005 | Loss 1.7971 | Val.Loss 1.8837 | Time(s) 0.1743
Epoch 00006 | Loss 1.7355 | Val.Loss 1.8731 | Time(s) 0.2178
Epoch 00007 | Loss 1.6957 | Val.Loss 1.8621 | Time(s) 0.2328
Epoch 00008 | Loss 1.7506 | Val.Loss 1.8507 | Time(s) 0.2230
Epoch 00009 | Loss 1.6602 | Val.Loss 1.8394 | Time(s) 0.2222
Epoch 00010 | Loss 1.6732 | Val.Loss 1.8282 |

## Evaluation

In [100]:
from sklearn.metrics import accuracy_score as acc
net.eval() # Set net to evaluation mode (deactivates dropout)
final_prediction = net(g, features).detach()

pred_sets = {"All ":final_prediction,"Train":final_prediction[mask],"Test":final_prediction[1-mask]}
label_sets = {"All ":labels,"Train":labels[mask],"Test":labels[1-mask]}
eval_functions = {"NLL-Loss":lambda y,x: F.nll_loss(x,y),"Accuracy":lambda y,x: acc(y,x.numpy().argmax(axis=1))}

for name,func in eval_functions.items():
    eval_message = f"\n{name}:\n"
    for subset in pred_sets.keys():
        eval_message += f" {subset}: {func(label_sets[subset],pred_sets[subset]):.4f} |"
    print(eval_message)


NLL-Loss:
 All : 1.3201 | Train: 0.6815 | Test: 1.3331 |

Accuracy:
 All : 0.5768 | Train: 1.0000 | Test: 0.5682 |
